In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import acquire
import prepare

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
rawdf = acquire.get_zillow_data()

In [4]:
rawdf.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount
0,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",NaN,06037,40.00,"298,371.00","3,714.58"
1,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",NaN,06037,67.00,"161,897.00","2,031.39"
2,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22.00,"472,384.00","7,234.74"
3,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",NaN,06037,77.00,"58,693.00",751.66
4,"-118,306261","33,745855",4.00,2.00,"1,329.00","7,005.00",NaN,06037,73.00,"62,194.00",886.96


In [5]:
df = prepare.prep_zillow(rawdf)

In [6]:
df.shape

(46889, 23)

In [7]:
df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",0.00,06037,40,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
1,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",0.00,06037,67,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
2,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
3,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",0.00,06037,77,"58,693.00",...,small,1,0,0,1,0,0,1,0,0
4,"-118,306261","33,745855",4.00,2.00,"1,329.00","7,005.00",0.00,06037,73,"62,194.00",...,small,1,0,0,0,1,0,1,0,0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

In [8]:
scaled_df = df.copy()

In [9]:
scaler = MinMaxScaler()

In [9]:
# scaler = sklearn.preprocessing.RobustScaler()

In [9]:
# # Quantile scaler
# qt = sklearn.preprocessing.QuantileTransformer(n_quantiles=10, output_distribution='normal', random_state=0)

In [10]:
# qt.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

QuantileTransformer(n_quantiles=10, output_distribution='normal',
                    random_state=0)

In [11]:
# scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = qt.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [10]:
scaler.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

MinMaxScaler()

In [11]:
scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = scaler.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [12]:
scaled_df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,217048","33,798657",0.60,0.18,0.13,0.05,0.00,06037,0.28,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
1,"-118,531253","34,039467",0.20,0.18,0.12,0.11,0.00,06037,0.48,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
2,"-117,613897","33,663005",0.80,0.36,0.40,0.06,1.00,06059,0.15,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
3,"-118,393309","34,227422",0.20,0.00,0.08,0.06,0.00,06037,0.55,"58,693.00",...,small,1,0,0,1,0,0,1,0,0
4,"-118,306261","33,745855",0.60,0.18,0.13,0.06,0.00,06037,0.52,"62,194.00",...,small,1,0,0,0,1,0,1,0,0


In [13]:
s_train, s_validate, s_test = prepare.split(scaled_df)

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
cols = ['square_feet', 'lot_size', 'has_pool', 'age', 'county_Los Angeles', 'county_Orange', 'bedrooms', 'bathrooms']

X_train = s_train[cols]
y_train = s_train.assessed_value

X_validate = s_validate[cols]
y_validate = s_validate.assessed_value

X_test = s_test[cols]
y_test = s_test.assessed_value

In [16]:
train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [84]:
final_train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
final_validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 
final_test_predictions = pd.DataFrame({
    'actual': s_test.assessed_value
})

## Multiple  Regression + RFE

In [18]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
k = 6

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=5)
rfe.fit(X_train, y_train)
print('selected top 8 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 8 features: Index(['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms'], dtype='object')


### Looping through k

In [19]:
for k in range(2,8):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    k+=1

In [20]:
train_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)"
40136,"391,960.00","616,327.55","663,399.13","646,554.58","645,985.80","676,169.17","700,473.61"
3252,"605,000.00","295,062.06","288,149.63","297,795.49","298,488.57","282,605.61","278,743.99"
16376,"1,139,000.00","756,778.75","743,093.58","759,505.21","759,181.58","778,780.40","796,795.49"
11892,"114,059.00","326,705.33","325,196.16","330,595.41","327,594.38","290,587.23","287,832.67"
11185,"645,000.00","368,245.50","373,798.34","373,662.09","374,012.99","419,119.02","413,315.88"


In [21]:
train_predictions['baseline'] = y_train.mean()

In [22]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   227,162.15
(multiple_rfe_k=, 6)   227,538.24
(multiple_rfe_k=, 5)   229,296.04
(multiple_rfe_k=, 4)   229,302.20
(multiple_rfe_k=, 3)   229,761.98
(multiple_rfe_k=, 2)   231,507.29
baseline               268,220.89
dtype: float64

In [23]:
validate_predictions['baseline'] = y_validate.mean()

In [24]:
validate_predictions.head()


,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)",baseline
34717,"657,000.00","352,216.76","397,112.85","404,487.99","404,068.52","382,078.19","378,406.48","404,167.11"
37478,"798,586.00","752,708.12","781,756.76","786,853.08","787,817.37","760,678.25","775,644.81","404,167.11"
10005,"432,366.00","375,297.16","339,511.16","333,650.72","333,638.35","308,499.54","302,707.93","404,167.11"
2443,"211,080.00","338,414.52","296,668.03","295,332.57","295,855.51","276,273.51","270,357.58","404,167.11"
39228,"248,333.00","227,049.95","250,639.37","243,680.44","241,598.56","278,832.42","280,334.97","404,167.11"


In [25]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   228,490.61
(multiple_rfe_k=, 6)   228,759.34
(multiple_rfe_k=, 5)   230,663.35
(multiple_rfe_k=, 4)   230,674.41
(multiple_rfe_k=, 3)   231,149.31
(multiple_rfe_k=, 2)   232,386.82
baseline               268,894.21
dtype: float64

In [26]:
# k=7 is the best

228490-227162

1328

**K=7 has the lowest rmse, highest performance**

In [86]:
# Adding k=7 to final table
lm = LinearRegression()
 # 1. Transform our X
    
rfe = RFE(lm, n_features_to_select=7)
rfe.fit(X_train, y_train)
# 2. Use the transformed x in our model
X_train_rfe = rfe.transform(X_train)
X_validate_rfe = rfe.transform(X_validate)
X_test_rfe = rfe.transform(X_test)
lm.fit(X_train_rfe, y_train)
    
# Make predictions
final_train_predictions['multiple_rfe_k=7'] = lm.predict(X_train_rfe)
final_validate_predictions['multiple_rfe_k=7'] = lm.predict(X_validate_rfe)
final_test_predictions['multiple_rfe_k=7']=lm.predict(X_test_rfe)

## Polynomial Features

In [26]:
train_pred = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_pred = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [27]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [28]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2
40136,"391,960.00","402,520.26","654,303.73"
3252,"605,000.00","402,520.26","290,744.50"
16376,"1,139,000.00","402,520.26","891,586.10"
11892,"114,059.00","402,520.26","206,289.57"
11185,"645,000.00","402,520.26","350,812.70"


In [29]:
validate_pred.head()

,actual,baseline,polynomial degree 2
34717,"657,000.00","404,167.11","344,361.55"
37478,"798,586.00","404,167.11","762,988.00"
10005,"432,366.00","404,167.11","290,992.62"
2443,"211,080.00","404,167.11","277,518.82"
39228,"248,333.00","404,167.11","268,137.72"


In [30]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)

In [31]:
train_rmse_b = mean_squared_error(train_pred.actual,train_pred['baseline'], squared = False)
validate_rmse_b = mean_squared_error(validate_pred.actual,validate_pred['baseline'], squared = False)

In [32]:
train_rmse, validate_rmse

(220526.60861779936, 221704.3813491752)

In [33]:
validate_rmse-train_rmse

1177.7727313758514

In [34]:
train_rmse_b, validate_rmse_b

(268220.89471220394, 268894.21404952626)

In [35]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
40136,"391,960.00","402,520.26","654,303.73","657,355.97"
3252,"605,000.00","402,520.26","290,744.50","292,351.20"
16376,"1,139,000.00","402,520.26","891,586.10","881,009.40"
11892,"114,059.00","402,520.26","206,289.57","212,273.45"
11185,"645,000.00","402,520.26","350,812.70","345,769.21"


In [36]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse

(220785.5974969383, 222197.98566867987)

In [37]:
validate_rmse-train_rmse

1412.3881717415643

In [38]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [39]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3
40136,"391,960.00","402,520.26","654,303.73","657,355.97","613,698.59"
3252,"605,000.00","402,520.26","290,744.50","292,351.20","286,671.88"
16376,"1,139,000.00","402,520.26","891,586.10","881,009.40","898,578.92"
11892,"114,059.00","402,520.26","206,289.57","212,273.45","206,023.67"
11185,"645,000.00","402,520.26","350,812.70","345,769.21","347,243.68"


In [40]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse

(218127.2626128267, 220900.0580065712)

In [41]:
# k=4
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [42]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 4'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 4'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4
40136,"391,960.00","402,520.26","654,303.73","657,355.97","613,698.59","559,725.41"
3252,"605,000.00","402,520.26","290,744.50","292,351.20","286,671.88","283,815.44"
16376,"1,139,000.00","402,520.26","891,586.10","881,009.40","898,578.92","923,946.58"
11892,"114,059.00","402,520.26","206,289.57","212,273.45","206,023.67","213,746.70"
11185,"645,000.00","402,520.26","350,812.70","345,769.21","347,243.68","343,688.26"


In [43]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(216035.16479299436, 226785.16303218642)

In [44]:
# k=5
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [45]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 5'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 5'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4,polynomial degree 5
40136,"391,960.00","402,520.26","654,303.73","657,355.97","613,698.59","559,725.41","553,179.69"
3252,"605,000.00","402,520.26","290,744.50","292,351.20","286,671.88","283,815.44","285,341.61"
16376,"1,139,000.00","402,520.26","891,586.10","881,009.40","898,578.92","923,946.58","954,109.73"
11892,"114,059.00","402,520.26","206,289.57","212,273.45","206,023.67","213,746.70","186,415.18"
11185,"645,000.00","402,520.26","350,812.70","345,769.21","347,243.68","343,688.26","340,815.24"


In [46]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(216035.16479299436, 226785.16303218642)

**K=3 has the lowest rmse**

In [88]:
# Adding k=3 to final table
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
final_train_predictions['baseline'] = y_train.mean()
final_validate_predictions['baseline'] = y_validate.mean()
final_test_predictions['baseline'] = y_test.mean()
final_train_predictions['polynomial degree 3'] = lm.predict(X_train_poly)
final_validate_predictions['polynomial degree 3'] = lm.predict(X_validate_poly)
final_test_predictions['polynomial degree 3'] = lm.predict(X_test_poly)

## Lasso-Lars

In [47]:
from sklearn.linear_model import LassoLars
# create the model object
lars = LassoLars(alpha=0)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)

pd.Series(lars.coef_, index=X_train.columns).sort_values()

bedrooms              -246,716.09
lot_size              -169,365.62
county_Los Angeles     -55,129.46
county_Orange           12,023.31
has_pool                34,839.69
age                     75,023.00
bathrooms              189,541.59
square_feet          1,172,709.64
dtype: float64

In [48]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars

In [49]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse

(227142.4885012844, 228470.26607945873)

In [50]:
validate_rmse-train_rmse

1327.7775781743403

In [90]:
# Adding to final table
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
X_test_pred_lars = lars.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['lasso_lars'] = X_train_pred_lars
final_validate_predictions['lasso_lars'] = X_validate_pred_lars
final_test_predictions['lasso_lars'] = X_test_pred_lars

## Generalized Linear Model

In [51]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm

In [52]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['glm'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['glm'], squared = False)
train_rmse, validate_rmse

(229784.92306563476, 232408.9133429264)

In [53]:
validate_rmse-train_rmse

2623.990277291625

In [91]:
# Adding to final table

glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
X_test_predict_glm = glm.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['glm'] = X_train_predict_glm
final_validate_predictions['glm'] = X_validate_predict_glm
final_test_predictions['glm'] = X_test_predict_glm

## Evaluation

In [94]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_train_predictions.actual, y_predicted, squared = False)

final_train_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   223,633.46
lasso_lars            232,745.23
multiple_rfe_k=7      232,763.09
glm                   235,321.90
baseline              274,862.72
dtype: float64

In [95]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_validate_predictions.actual, y_predicted, squared = False)

final_validate_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   222,070.92
lasso_lars            230,203.61
multiple_rfe_k=7      230,221.87
glm                   233,617.59
baseline              272,468.83
dtype: float64

In [96]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_test_predictions.actual, y_predicted, squared = False)

final_test_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   225,323.49
lasso_lars            233,614.86
multiple_rfe_k=7      233,644.57
glm                   237,682.56
baseline              277,387.85
dtype: float64

In [97]:
(277387.85-225323.49)/277387.85

0.18769517121964782